In [19]:
import keras
from tablut.game import Board
import random
import numpy as np
from tqdm import tqdm
from keras.models import Model
from keras.layers import *
import tensorflow as tf
import matplotlib.pyplot as plt

In [8]:
board = Board()
pos_vec = board.to_network_input()

In [17]:
inp = Input((252,))

l1 = Dense(128, activation='relu')(inp)
l2 = Dense(128, activation='relu')(l1)
l3 = Dense(128, activation='relu')(l2)
l4 = Dense(128, activation='relu')(l3)
l5 = Dense(128, activation='relu')(l4)

policyOut = Dense(1215, name='policyHead', activation='softmax')(l5)
valueOut = Dense(1, activation='tanh', name='valueHead')(l5)

bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model = Model(inp, [policyOut,valueOut])
model.compile(optimizer = 'SGD', loss={'valueHead' : 'mean_squared_error', 'policyHead' : bce})

In [18]:
def fst(a):
    return a[0]

# white == random player
# black == network
def net_vs_rand(board):
    record = []
    while(not fst(board.is_terminal())):
        if(board.turn == Board.WHITE):
            moves = board.generate_moves()
            m = moves[random.randint(0, len(moves)-1)]
            board.apply_move(m)
            record.append(m)
            continue
        else:
            q = model.predict(np.array([board.to_network_input()]))
            masked_output = [0 for x in range(0,1216)]
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                masked_output[m_idx] = q[0][0][m_idx]
            best_idx = np.argmax(masked_output)
            sel_move = None
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                if(best_idx == m_idx):
                    sel_move = m
            board.apply_move(sel_move)
            record.append(sel_move)
            continue
    terminal, winner = board.is_terminal()
    return winner

# white random player
# black random player
def rand_vs_rand(board):
    while(not fst(board.is_terminal())):
        moves = board.generate_moves()
        m = moves[random.randint(0, len(moves)-1)]
        board.apply_move(m)
        continue
    terminal, winner = board.is_terminal()
    return winner


white_wins = 0
black_wins = 0
draws = 0

for i in tqdm(range(0,100)):
    board = Board()
    board.set_starting_position()
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
    winner = net_vs_rand(board)
    if(winner == Board.WHITE):
        white_wins += 1
    if(winner == Board.BLACK):
        black_wins += 1
    if(winner == 0):
        draws += 1

all = white_wins + black_wins + draws
print("Rand Network vs Reinforcement: "+str(white_wins/all) + "/"+str(black_wins/all) + "/"+str(draws/all))


white_wins = 0
black_wins = 0

for i in tqdm(range(0,100)):
    board = Board()
    board.set_starting_position()
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
    winner = rand_vs_rand(board)
    if(winner == Board.WHITE):
        white_wins += 1
    if(winner == Board.BLACK):
        black_wins += 1
    if winner == 0:
        draws += 1

all = white_wins + black_wins
print("Rand vs Rand Network: "+str(white_wins/all) + "/"+str(black_wins/all)+ "/"+str(draws/all))

  0%|          | 0/100 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

In [20]:
def draw_board(board_state):
    """
    Draw the game board based on the given board state.
    """
    # Reshape the 1x81 list to a 9x9 list
    board_state = np.reshape(board_state, (9, 9))

    # Create a new plot
    fig, ax = plt.subplots()

    # Draw vertical lines
    for i in range(1, 9):
        plt.axvline(x=i, color='black', linewidth=2)

    # Draw horizontal lines
    for j in range(1, 9):
        plt.axhline(y=j, color='black', linewidth=2)

    # Draw the game board
    for i in range(9):
        for j in range(9):
            # Add piece if exists in the board state
            ax.text(i + 0.5, j + 0.5, str(i*9+j), color='grey', ha='center', va='center', fontsize=20, alpha=0.25)
            piece = board_state[i][j]
            if piece != 0:
                if piece == 2:
                    ax.text(i + 0.5, j + 0.5, 'B', color='black', ha='center', va='center', fontsize=20)
                elif piece == 1:
                    ax.text(i + 0.5, j + 0.5, 'W', color='red', ha='center', va='center', fontsize=20)
                else:
                    ax.text(i + 0.5, j + 0.5, 'K', color='orange', ha='center', va='center', fontsize=20)

    # Set the aspect of the plot to equal
    ax.set_aspect('equal')

    # Set the limits of the plot
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 9)

    # Remove the axis
    ax.axis('off')

    # Show the plot
    plt.show()

In [25]:
board = Board()
board.set_starting_position()

In [28]:
board.generate_moves()

In [27]:
if(board.turn == Board.WHITE):
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
else:
    q = model.predict(np.array([board.to_network_input()]))
    masked_output = [0 for x in range(0,1216)]
    for m in board.generate_moves():
        m_idx = board.get_network_output_index(m)
        masked_output[m_idx] = q[0][0][m_idx]
    best_idx = np.argmax(masked_output)
    sel_move = None
    for m in board.generate_moves():
        m_idx = board.get_network_output_index(m)
        if(best_idx == m_idx):
            sel_move = m
    board.apply_move(sel_move)

draw_board(board.board)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


TypeError: 'NoneType' object is not iterable